In [1]:
import truecase
import os
import json
import pickle
import globals
import transformers
#import dataset
import torch
import pandas as pd
from sklearn import model_selection
import engine
import numpy as np
#from model import BERTBaseUncased
from transformers import AdamW, get_linear_schedule_with_warmup
# from transformers import get_linear_schedule_with_warmup
from sklearn import metrics
import torch.nn as nn
from transformers import BertTokenizer, BertConfig, BertModel

In [2]:
TRAINING_FILE = "/Users/zhangzihan/Documents/pycharm_project/graduation_project/data/phase1_relevant_fact_selection_trainset.csv"
tokenizer = BertTokenizer.from_pretrained("/Users/zhangzihan/Documents/pycharm_project/graduation_project/data/bert-base-uncased-vocab.txt",
                                               do_lower_case = False)

/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [3]:
dfx = pd.read_csv(TRAINING_FILE).fillna("none")
dfx = dfx.iloc[0:3000]
# df_train, df_valid = model_selection.train_test_split(dfx, test_size=0.2, random_state=42,
#                                                           stratify=dfx.label.values)
# dfx = dfx.iloc[1999:2119]
df_train, df_valid = model_selection.train_test_split(dfx, test_size=0.2, random_state=42,
                                                          stratify=dfx.label.values)
print(len(df_train))
print(len(df_valid))

2400
600


In [4]:
# dfx = pd.read_csv(TRAINING_FILE).fillna("none")
# df_train, df_valid = model_selection.train_test_split(dfx, test_size=0.2, random_state=42,
#                                                           stratify=dfx.label.values)
# # row

# print(df_train.shape[0])
# print(df_valid.shape[0])
# print(df_train)

In [5]:
# 转化为Dataset类型？
# Dataset是一个包装类，用来将数据包装为Dataset类，然后传入DataLoader中，再使用DataLoader这个类来更加快捷的对数据进行操作。

class BERTDataset:
    def __init__(self, q1, q2, target):
        self.q1 = q1
        self.q2 = q2
        self.target = target
        self.tokenizer = tokenizer
        self.max_len = 512
    def __len__(self):
        return len(self.q1)
    def __getitem__(self, item):
        q1 = str(self.q1[item])
        q2 = str(self.q2[item])
        q1 = " ".join(q1.split())
        q2 = " ".join(q2.split())
        #’input_ids:是单词在词典中的编码 ‘token_type_ids’:区分两个句子的编码（上句全为0，下句全为1）‘attention_mask’:指定对哪些词进行self-Attention操作
        inputs = self.tokenizer.encode_plus(q1, q2, add_special_tokens=True, 
                                            max_length=self.max_len, 
                                            padding='longest',
                                            truncation=True)
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        padding_length = self.max_len - len(ids)
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)

        return {'ids': torch.tensor(ids, dtype=torch.long), 'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'targets': torch.tensor(self.target[item], dtype=torch.long)}

In [6]:
# pass the q1,q2,target here .
TRAIN_BATCH_SIZE = 50
VALID_BATCH_SIZE = 50
train_dataset = BERTDataset(df_train.question.values, df_train.context.values, target=df_train.label.values)

# 从Dataset类型转换为Dataloader类型？<question, answer, label>
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

valid_dataset = BERTDataset(df_valid.question.values, df_valid.context.values, df_valid.label.values)

valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1)

In [7]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained("/Users/zhangzihan/Documents/pycharm_project/graduation_project/data/bert-base-uncased")
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 1)

    def forward(self, ids, mask, token_type_ids):
        outs = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo = self.bert_drop(outs.pooler_output)
        output = self.out(bo)
        return output

In [8]:
device = torch.device("cpu")
model = BERTBaseUncased()
# model = transformers.BertModel.from_pretrained("/Users/zhangzihan/Documents/pycharm_project/graduation_project/data/bert-base-uncased", 
#                                                       num_labels=2)
model.to(device)

BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [9]:
# name_parameters()给出网络层的名字和参数的迭代器
# named_parameters()返回的list中，每个元祖打包了2个内容，分别是layer-name和layer-param
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayetNorm.weight']
optimizer_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.1}]


In [10]:
EPOCHS = 2
num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
print("\nnum_train_steps:")
print(num_train_steps)
model = nn.DataParallel(model)
best_accuracy = 0
MODEL_PATH = "/Users/zhangzihan/Documents/pycharm_project/graduation_project/model/model.bin"
for epoch in range(EPOCHS):
    engine.train_fn(train_data_loader, model, optimizer, device, scheduler)
    outputs, targets = engine.eval_fn(valid_data_loader, model, device)

    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    print("Accuracy Score = ", accuracy)

    if accuracy > best_accuracy:
        print("Saving Checkpoint!!!")
        torch.save(model.state_dict(), MODEL_PATH)
        best_accuracy = accuracy


num_train_steps:
96


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
100%|███████████████████████████████████████████| 12/12 [13:27<00:00, 67.27s/it]


Accuracy Score =  0.8433333333333334
Saving Checkpoint!!!


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
 10%|████▎                                    | 5/48 [23:14<3:19:56, 278.99s/it]


KeyboardInterrupt: 